In [3]:
import json
import pandas as pd

# load data
with open('../data/municipality_borders.geojson', 'r') as geojson_file:
    geojson_data = json.load(geojson_file)

csv_data = pd.read_csv('../data/population_data.csv')

In [4]:
# inspect the first few rows of the CSV and the structure of one GeoJSON feature
csv_head = csv_data.head()
geojson_feature_example = geojson_data['features'][0] if geojson_data['features'] else None

csv_head, geojson_feature_example

(  BFS;GEMEINDE;Bevoelkerung_20-64
 0              131;Adliswil;12120
 1             241;Aesch (ZH);1011
 2          1;Aeugst am Albis;1210
 3       2;Affoltern am Albis;7731
 4                 211;Altikon;432,
 {'type': 'Feature',
  'properties': {'BFS': 117,
   'NAME': 'Hinwil',
   'BEZIRKSNAM': 'Hinwil',
   'ART_TEXT': 'Gemeinde',
   'ART_CODE': 1},
  'geometry': {'type': 'Polygon',
   'coordinates': [[[8.847781115143963, 47.32409547958538],
     [8.85860959968332, 47.321623363987094],
     [8.865388842573168, 47.323014843191096],
     [8.872682282932749, 47.31815598568517],
     [8.886942790747097, 47.31663449079118],
     [8.889044678155368, 47.31788204327347],
     [8.894165445833671, 47.313092892419405],
     [8.896102489121994, 47.31293418058575],
     [8.899114775013048, 47.31314595149962],
     [8.900187385514435, 47.31222170908761],
     [8.907614348731368, 47.31143183182875],
     [8.90795150130765, 47.311478068673274],
     [8.905804611637135, 47.30894930053905],
     [8.8

In [5]:
# Parse the CSV data correctly, splitting the column into BFS, Gemeinde (municipality), and population
csv_data_parsed = csv_data['BFS;GEMEINDE;Bevoelkerung_20-64'].str.split(';', expand=True)
csv_data_parsed.columns = ['BFS', 'GEMEINDE', 'Population']
csv_data_parsed['BFS'] = csv_data_parsed['BFS'].astype(int)

csv_data_parsed.head()

,BFS,GEMEINDE,Population
0,131,Adliswil,12120
1,241,Aesch (ZH),1011
2,1,Aeugst am Albis,1210
3,2,Affoltern am Albis,7731
4,211,Altikon,432


In [7]:
# update the municipality geojson data with the corresponding population data

for feature in geojson_data['features']:
    bfs_number = feature['properties'].get('BFS')
    population_row = csv_data_parsed[csv_data_parsed['BFS'] == bfs_number]
    if not population_row.empty:
        feature['properties']['Population'] = int(population_row['Population'].values[0])

updated_geojson_path = '../data/municipality_borders_population_data.geojson'
with open(updated_geojson_path, 'w') as updated_geojson_file:
    json.dump(geojson_data, updated_geojson_file)

updated_geojson_path


'../data/municipality_borders_population_data.geojson'